In [872]:
import numpy as np
from math import floor

import os
import glob
import time

import torch
print(torch.__version__)
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
from pyts.image import GramianAngularField
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import yfinance as yf

#set gpu env
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device",device)
print("cuda version",torch.version.cuda)

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())

2.3.0+cu121
Device cuda:0
cuda version 12.1
CUDA Available: True
CUDA Device Count: 1


In [873]:
class ToyLossLayer:
    """
    Computes square loss with first element of hidden layer array.
    """
    @classmethod
    def loss(self, pred, label):
        pred = pred.to(label.device)
        loss = (pred[0][0] - label) ** 2
        #print("ToyLossLayer pred shape",pred.shape,"ToyLossLayer pred[0] shape",pred[0].shape,"label shape",label.shape,"loss shape",loss.shape,"VALUE pred",pred[0][0],"VALUE LABEL",label)
        print("LOSS:",loss.shape,"pred val=",pred[0][0],"LABEL val=",label)
        return loss

    @classmethod
    def bottom_diff(self, pred, label):
        pred = pred.to(label.device)
        diff = torch.zeros_like(pred)
        diff[0] = 2 * (pred[0][0] - label)
        #print("bottom_diff",diff[0],"pred",pred[0][0],"label",label)
        print("pred[0][0]",pred[0][0],"label",label)
        return diff

In [874]:
def sigmoid(x):
    return torch.sigmoid(x)

def sigmoid_derivative(values):
    # Sigmoid derivative: sigmoid(x) * (1 - sigmoid(x))
    sigmoid_values = torch.sigmoid(values)
    return sigmoid_values * (1 - sigmoid_values)

def tanh_derivative(values):
    # Tanh derivative: 1 - tanh(x)^2
    return 1.0 - torch.tanh(values) ** 2

def rand_arr(a, b, *args, device='cpu'):
    # Create a random tensor on the specified device (CPU or CUDA)
    return (torch.rand(*args, device=device) * (b - a)) + a

Create Simple LSTM, Backpropagate without Libraries

In [875]:
import torch.nn.init as init

class LstmParam:
    def __init__(self, mem_cell_ct, x_dim):
        self.device = torch.device(device)
        self.mem_cell_ct = mem_cell_ct
        self.x_dim = x_dim
        #concat to accomodate matrix multiplication of input + memcell by weights
        concat_len = x_dim + mem_cell_ct
        # weight matrices
        self.wg = torch.empty((mem_cell_ct, concat_len), device=self.device)
        self.wi = torch.empty((mem_cell_ct, concat_len), device=self.device)
        self.wf = torch.empty((mem_cell_ct, concat_len), device=self.device)
        self.wo = torch.empty((mem_cell_ct, concat_len), device=self.device)
        init.uniform_(self.wg, -0.1, 0.1)
        init.uniform_(self.wi, -0.1, 0.1)
        init.uniform_(self.wf, -0.1, 0.1)
        init.uniform_(self.wo, -0.1, 0.1)
        # bias terms
        self.bg = torch.empty((mem_cell_ct), device=self.device)
        self.bi = torch.empty((mem_cell_ct), device=self.device)
        self.bf = torch.empty((mem_cell_ct), device=self.device) 
        self.bo = torch.empty((mem_cell_ct), device=self.device)
        init.uniform_(self.bg, -0.1, 0.1)
        init.uniform_(self.bi, -0.1, 0.1)
        init.uniform_(self.bf, -0.1, 0.1)
        init.uniform_(self.bo, -0.1, 0.1)
        # diffs (derivative of loss function w.r.t. all parameters)
        self.wg_diff = torch.zeros_like(self.wg)
        self.wi_diff = torch.zeros_like(self.wi)
        self.wf_diff = torch.zeros_like(self.wf)
        self.wo_diff = torch.zeros_like(self.wo)
        self.bg_diff = torch.zeros_like(self.bg)
        self.bi_diff = torch.zeros_like(self.bi)
        self.bf_diff = torch.zeros_like(self.bf)
        self.bo_diff = torch.zeros_like(self.bo)

    def apply_diff(self, lr = 1):
        self.wg -= lr * self.wg_diff
        self.wi -= lr * self.wi_diff
        self.wf -= lr * self.wf_diff
        self.wo -= lr * self.wo_diff
        self.bg -= lr * self.bg_diff
        self.bi -= lr * self.bi_diff
        self.bf -= lr * self.bf_diff
        self.bo -= lr * self.bo_diff
        # reset diffs to zero
        self.wg_diff.zero_()
        self.wi_diff.zero_()
        self.wf_diff.zero_()
        self.wo_diff.zero_()
        self.bg_diff.zero_()
        self.bi_diff.zero_()
        self.bf_diff.zero_()
        self.bo_diff.zero_()

In [876]:
class LstmState:
    def __init__(self, mem_cell_ct, x_dim):
        self.device = torch.device(device)
        self.mem_cell_ct = mem_cell_ct
        #print("***********device",self.device)
        self.g = torch.zeros(mem_cell_ct, device=self.device) #input node (tanh xt and ht-1)
        self.i = torch.zeros(mem_cell_ct, device=self.device) #input gate (sigmoid xt and ht-1, has gate and multiplies input node)
        self.f = torch.zeros(mem_cell_ct, device=self.device) #forget gate
        self.o = torch.zeros(mem_cell_ct, device=self.device) #output gate
        self.s = torch.zeros(mem_cell_ct, device=self.device) #internal state
        self.h = torch.zeros(mem_cell_ct, device=self.device) #value in hidden layers
        self.bottom_diff_h = torch.zeros_like(self.h)
        #print("***At init bottom_diff_h",self.bottom_diff_h.shape)
        self.bottom_diff_s = torch.zeros_like(self.s)

In [877]:
class LstmNode:
    def __init__(self, lstm_param, lstm_state):
        # store reference to parameters and to activations
        self.state = lstm_state
        self.param = lstm_param
        # non-recurrent input concatenated with recurrent input
        self.xc = None

    def bottom_data_is(self, x, s_prev = None, h_prev = None):
        #if this is the first lstm node in the network
        #Convert to cuda only necessary: x, s_prev, and h_prev if they are already on CUDA
        x = x.to(self.state.device)
        s_prev = s_prev.to(self.state.device) if s_prev is not None else torch.zeros_like(self.state.s)
        h_prev = h_prev.to(self.state.device) if h_prev is not None else torch.zeros_like(self.state.h)

        # save data for use in backprop
        self.s_prev = s_prev
        self.h_prev = h_prev

        x = x.view(x.size(0), -1)

        # concatenate x(t) and h(t-1)
        #print("h_prev shape before squeeze",h_prev.shape)
        if h_prev.dim() != 2:
            h_prev = h_prev.unsqueeze(0)
        #print("x shape",x.shape,"h_prev shape after squeeze",h_prev.shape)
        xc = torch.cat((x, h_prev), dim=1)  # Assuming concatenating along the feature dimension
        self.xc = xc
        #print("*********ENTRY xc.shape",self.xc.shape)

        #print("self.param.wg shape",self.param.wg.shape,"xc shape",xc.shape,"xc shape after unsqueeze",xc.unsqueeze(1).shape,"self.param.bg shape",self.param.bg.shape)
        #print("self.param.wg shape",self.param.wg.shape,"self.param.wg.T shape",self.param.wg.T.shape,"xc shape",xc.shape,"self.param.bg shape",self.param.bg.shape)
        self.state.g = torch.tanh(torch.mm(xc, self.param.wg.T).squeeze(1) + self.param.bg) #candidate cell state Ct
        self.state.i = torch.sigmoid(torch.mm(xc, self.param.wi.T).squeeze(1) + self.param.bi)
        self.state.f = torch.sigmoid(torch.mm(xc, self.param.wf.T).squeeze(1) + self.param.bf)
        self.state.o = torch.sigmoid(torch.mm(xc, self.param.wo.T).squeeze(1) + self.param.bo)
        
        # cell state (state.s) carries LT dependencies and modified mininally each step due to gating
        # self.state.s is next s_prev
        self.state.s = self.state.g * self.state.i + s_prev * self.state.f
        # hidden state is the node state , carries ST dependencies
        # self.state.h is next h_prev
        self.state.h = torch.tanh(self.state.s) * self.state.o
    
    def top_diff_is(self, top_diff_h, top_diff_s):
        # No need to convert top_diff_h and top_diff_s if they are already on CUDA
        top_diff_h = top_diff_h.to(self.state.device)
        top_diff_s = top_diff_s.to(self.state.device)

        # notice that top_diff_s is carried along the constant error carousel
        ds = self.state.o * top_diff_h + top_diff_s
        do = self.state.s * top_diff_h
        di = self.state.g * ds
        dg = self.state.i * ds
        df = self.s_prev * ds

        # diffs w.r.t. vector inside sigma / tanh function
        #print();print("***shape state i",self.state.i.shape,"shape di",di.shape)
        di_input = sigmoid_derivative(self.state.i) * di 
        df_input = sigmoid_derivative(self.state.f) * df 
        do_input = sigmoid_derivative(self.state.o) * do 
        dg_input = tanh_derivative(self.state.g) * dg

        # diffs w.r.t. inputs
        # Ensure di_input, df_input, do_input, dg_input are 2-D before using torch.mm
        di_input = di_input.view(1, -1)  # Ensure it's a 2-D tensor
        df_input = df_input.view(1, -1)
        do_input = do_input.view(1, -1)
        dg_input = dg_input.view(1, -1)
 
        #print();print("**************di_input shape:", di_input.T.shape,"*************self.xc shape:", self.xc.shape)
        
        self.param.wi_diff += torch.mm(di_input.T, self.xc)
        self.param.wf_diff += torch.mm(df_input.T, self.xc)
        self.param.wo_diff += torch.mm(do_input.T, self.xc)
        self.param.wg_diff += torch.mm(dg_input.T, self.xc)

        self.param.bi_diff += di_input.squeeze(0)
        self.param.bf_diff += df_input.squeeze(0)
        self.param.bo_diff += do_input.squeeze(0)
        self.param.bg_diff += dg_input.squeeze(0)

        # Compute bottom diff
        dxc = torch.zeros_like(self.xc)
        #print("#############**********BEFORE shape dxc",dxc.shape,"self.xc shape",self.xc.shape)
        dxc += torch.mm(self.param.wi.T, di_input.T).squeeze(1)
        dxc += torch.mm(self.param.wf.T, df_input.T).squeeze(1)
        dxc += torch.mm(self.param.wo.T, do_input.T).squeeze(1)
        dxc += torch.mm(self.param.wg.T, dg_input.T).squeeze(1)
        #print();print("#############**********AFTER dxc shape",dxc.shape)

        # Save bottom diffs
        #print();print("######BEFORE self.state.bottom_diff_h shape",self.state.bottom_diff_h.shape)
        self.state.bottom_diff_s = ds * self.state.f
        #print("=============xdim size",self.param.x_dim)
        self.state.bottom_diff_h = dxc[:,self.param.x_dim:]
        #print();print("######AFTER self.state.bottom_diff_s shape",self.state.bottom_diff_s.shape)

In [878]:
class LstmNetwork():
    def __init__(self, lstm_param):
        self.lstm_param = lstm_param
        self.lstm_node_list = []
        # input sequence
        self.x_list = []

    def y_list_is(self, y_list, loss_layer):
        """
        Updates diffs by setting target sequence with corresponding loss layer. 
        It doesn't update params, need to call self.lstm_param.apply_diff()
        """
        assert len(y_list) == len(self.x_list)
        idx = len(self.x_list) - 1

        #Convert y_list elements to CUDA tensors
        y_list = [torch.tensor(y, device=self.lstm_param.device) for y in y_list]

        state_h_numpy_flat_list = self.lstm_node_list[idx].state.h.cpu().detach().numpy().flatten().tolist()
        
        print("***To Calc Loss at idx",idx,"state.h count",len(state_h_numpy_flat_list),"state.h at node IDX",self.lstm_node_list[idx].state.h,"===y_list[idx]",y_list[idx].cpu().detach().numpy(), "y_list",y_list)

        # first node only gets diffs from label ...
        loss = loss_layer.loss(self.lstm_node_list[idx].state.h, y_list[idx])
        print("BEFORE idx>=0 Node LOSS: idx",idx,"self.lstm_node_list[idx].state.h",self.lstm_node_list[idx].state.h,"y_list[idx]",y_list[idx])
        diff_h = loss_layer.bottom_diff(self.lstm_node_list[idx].state.h, y_list[idx])
        # here s is not affecting loss due to h(t+1), hence we set equal to zero
        diff_s = torch.zeros(self.lstm_param.mem_cell_ct, device=self.lstm_param.device)
        self.lstm_node_list[idx].top_diff_is(diff_h, diff_s)
        idx -= 1

        ### ... following nodes also get diffs from next nodes, hence we add diffs to diff_h
        ### we also propagate error along constant error carousel using diff_s
        while idx >= 0:
            print("AFTER idx>=0 Node LOSS: idx",idx,"self.lstm_node_list[idx].state.h",self.lstm_node_list[idx].state.h,"y_list[idx]",y_list[idx])
            loss += loss_layer.loss(self.lstm_node_list[idx].state.h, y_list[idx])
            diff_h = loss_layer.bottom_diff(self.lstm_node_list[idx].state.h, y_list[idx])
            #print("idx",idx,"shape diff_h",diff_h.shape,"len self.lstm_node_list",len(self.lstm_node_list),"shape self.lstm_node_list[idx].state.h",self.lstm_node_list[idx].state.h.shape,"shape bottom_diff_h ",self.lstm_node_list[idx + 1].state.bottom_diff_h.shape)
            diff_h += self.lstm_node_list[idx + 1].state.bottom_diff_h
            diff_s = self.lstm_node_list[idx + 1].state.bottom_diff_s
            self.lstm_node_list[idx].top_diff_is(diff_h, diff_s)
            idx -= 1

        return loss

    def x_list_clear(self):
        self.x_list = []

    def x_list_add(self, x):
        #convert to cuda if necessary
        x = x if x.device == self.lstm_param.device else x.to(self.lstm_param.device)
        
        self.x_list.append(x)
        if len(self.x_list) > len(self.lstm_node_list):
            # need to add new lstm node, create new state mem
            lstm_state = LstmState(self.lstm_param.mem_cell_ct, self.lstm_param.x_dim)
            self.lstm_node_list.append(LstmNode(self.lstm_param, lstm_state))

        # get index of most recent x input
        idx = len(self.x_list) - 1
        print("initial idx",idx)
        if idx == 0:
            # no recurrent inputs yet
            # if x.device.type == 'cuda':
            #     print ("x in cuda")
            # else: 
            #     print("x not in cuda")
            self.lstm_node_list[idx].bottom_data_is(x)
            print("Adding to Node Inputs at idx",idx,"inputs:",x)
        else:
            s_prev = self.lstm_node_list[idx - 1].state.s
            #print("s_prev at idx",idx,"s_prev:",s_prev)
            h_prev = self.lstm_node_list[idx - 1].state.h
            #print("h_prev at idx",idx,"h_prev:",s_prev)
            self.lstm_node_list[idx].bottom_data_is(x, s_prev, h_prev)
            print("Adding to Node Inputs at idx",idx,"inputs:",x)

In [879]:
# Iterate Through Training Loop:

# For each iteration (total of 100 iterations), the function:
    # Adds each input vector to the LSTM network, each a node.
    # Each node has total num mem cells defined
    # Computes the loss for each node using the y_list_is function.
    # Applies the parameter updates.
    # Clears the input list of the LSTM network.

torch.set_printoptions(threshold=torch.inf)

def run_lstm_simple(train_loader):
    # learns to repeat simple sequence from random inputs
    np.random.seed(0)

    epochs = 100

    # parameters for input data dimension and lstm cell count
    mem_cell_ct = 100
    x_dim = 32*32
    lstm_param = LstmParam(mem_cell_ct, x_dim)
    lstm_net = LstmNetwork(lstm_param)
    #y_list = [-0.5, 0.2, 0.1, -0.5]

    for epoch in range(epochs):  
        print("epoch", "%2s" % str(epoch), end=": ")

        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            for ind in range(len(inputs)):
                #print("epoch",epoch,"ind",ind,"inputs[ind] size",inputs[ind].shape,"inputs[ind]",inputs[ind])
                print("epoch",epoch,"ind",ind,"labels[ind] size",labels[ind].shape,"labels[ind]",labels[ind])
                lstm_net.x_list_add(inputs[ind])

            #get all state.h from all nodes in lstm_node_list
            y_preds = [node.state.h[0].cpu().detach().numpy() for node in lstm_net.lstm_node_list]
            
            #flatten to single list
            y_preds_flat = [item for sublist in y_preds for item in sublist]
            print("epoch",epoch,"len y_preds_flat i.e. all nodes state.h",len(y_preds_flat),"pred = [" + ", ".join(["%2.5f" % pred for pred in y_preds_flat]) + "]", end=", ");print()
            print("epoch",epoch,"y_preds_flat[0] i.e. i.e. 1stNode.state.h =", y_preds_flat[0])
            #print("epoch",epoch,"labels =", labels)
            
            # Compute loss
            loss = lstm_net.y_list_is(labels, ToyLossLayer)
            print("loss:", "%.3e" % loss.item())  # Use .item() to get the scalar value from tensor
        
            #update weights and zero diffs
            lstm_param.apply_diff(lr=0.1)
            lstm_net.x_list_clear()

torch.set_printoptions()

Import and encode data to GADF

In [880]:
#close price time period
start_date = '2021-10-01'
#start_date = '2023-01-01'
end_date = '2023-12-01'

ticker = 'SIVBQ'
dataset = yf.download(ticker, start=start_date, end=end_date, interval='1d')

print("num rows",dataset.shape[0])

dataset.dropna(how='any', inplace=True)
#print("Num rows for df Close col",len(dataset['Close'].dropna()))
print("Num rows for df Close col",len(dataset['Close']))
print(dataset.head())

[*********************100%%**********************]  1 of 1 completed

num rows 524
Num rows for df Close col 524
                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2021-10-01  650.000000  669.570007  648.065002  664.530029  664.530029  281112
2021-10-04  663.989990  669.000000  645.729797  649.349976  649.349976  340741
2021-10-05  657.909973  669.174988  652.072571  665.380005  665.380005  358684
2021-10-06  654.039978  663.919983  643.219971  659.849976  659.849976  441251
2021-10-07  670.929993  679.000000  663.280029  665.559998  665.559998  258661


## Imaging Algorithm: Generate Gramian angular field (GAF) images from time series data 

In [881]:
def generate_gaf_images(dataset, gaf_img_sz=2, method="summation", sample_range=(0,1)):
    #print("len data series received:",len(dataset),"size",dataset.size)

    #determine num of gaf_img_szX images with gaf_img_sz datapoints
    num_images_to_generate = floor(len(dataset) / (gaf_img_sz*2))
    data_ele_per_img = gaf_img_sz*gaf_img_sz
    print("len dataset",len(dataset),"num_images_to_generate",num_images_to_generate)
    
    #reshape dataset into number of images
    dataset = dataset[:num_images_to_generate*(gaf_img_sz*gaf_img_sz)]
    print("data in GAF",dataset,"len dataset",len(dataset))
    
    price_list=[]
    for i in range(num_images_to_generate):
        start_index = i*data_ele_per_img
        price_list.append(dataset[start_index+1:start_index+1+data_ele_per_img])
        print("Num images to generate",num_images_to_generate,"dataset i",i,"len dataset i",len(dataset[start_index+1:start_index+1+data_ele_per_img]))
    #print("prices in GAF",price_list)
    #print("image_size",gaf_img_sz)
    
    gaf = GramianAngularField(image_size=gaf_img_sz, method=method, sample_range=sample_range)
    gaf_images= gaf.fit_transform(dataset)
    #print("gaf_image",gaf_images.shape)
    #print("returning price list",mean_price_list)
    
    return gaf_images, price_list

## Generate images for open, high, low, close and adj close prices

We transpose the resulting image list to represent:
+ 16: number of images
+ 5: number of image channels/features. Each image has 5 ="Open", "High", "Low", "Close", and "Adj Close"
+ 32: image height
+ 32: image width
+ the label is the 32+1 price for each time series chunk (image) corresponding to the relevant feature

In [882]:
np.set_printoptions(threshold=np.inf)

#cols_used = ["Open", "High", "Low", "Close", "Adj Close"]
cols_used = ["Close"]
cols_used_count = sum(column_name in cols_used for column_name in dataset.columns)

#TODO: Instead of insert a list in each list for each feature, insert the elements directly
def generate_multiple_feature_images(dataset, image_size=2, method="summation", sample_range = (0, 1)):
    
    feature_image_dataset_list=[[] for _ in range(len(cols_used))]
    feature_price_dataset_list=[[] for _ in range(len(cols_used))] #="Open", "High", "Low", "Close" , "Adj Close"
    feature_label_dataset_list=[] #next value for each chunk of ="Open", "High", "Low", "Close" , "Adj Close"
    column_idx = 0

    for idx, column_name in enumerate(dataset.columns):

      #create open, high, low and close images
      #if column_name in ["Open", "High", "Low", "Close", "Adj Close"]:
      if column_name in ["Close"]:
        temp_image_list = []
        temp_price_list = []
        temp_label_list = []
        #print("dataset idx", idx, "len rows this data feature", len(dataset[i]), "dataset[i].shape", dataset[i].shape, "dataset i:", dataset[i])

        full_feature_data = dataset[column_name].values
        #reduce by one to account for the last label
        full_feature_num_samples = len(full_feature_data-1)
        print("full_feature_num_samples",full_feature_num_samples)

        inputs_per_image = int(image_size*2)

        gaf_images, price_list = generate_gaf_images(full_feature_data, gaf_img_sz=image_size, method=method, sample_range=sample_range)
        temp_image_list.append(gaf_images)
        if(curr_input_start_index==0):
            print("Price Data Pre-Gaf: i", curr_input, "len",len(curr_input), "shape", curr_input.shape, "data",curr_input)
            print("Image Returned: column", idx, "image size", gaf_images.size, f"first {image_size} image vals", gaf_images.flatten())
        temp_price_list.append(price_list)
        temp_label_list.append(full_feature_data[curr_input_start_index + image_size + 1])

        for curr_input_start_index in range(0, full_feature_num_samples, inputs_per_image):
            #curr_input_index + inputs_per_image + 1 to account for label
            max_input_index = curr_input_start_index + inputs_per_image + 1
            if(max_input_index <= full_feature_num_samples):
                #exclude label from input
                curr_input = full_feature_data[curr_input_start_index:max_input_index -1]
            
                
                #feature_label_index_dataset_list.append(feature_data[cur_chunk + image_size + 1])
                #print("chunk",cur_chunk,"label for",column_name,"price",feature_data[cur_chunk + image_size + 1])
                #if(column_name == "Open"):
                #index position for the label of this chunk
                #feature_label_index_dataset_list.append(cur_chunk + image_size + 1)
                #print("at chunk",cur_chunk,"feature label list",feature_label_index_dataset_list)
            
            print(f"Column {column_name} temp image list len to append",len(temp_image_list))
            feature_image_dataset_list[column_idx].append(temp_image_list)
            #print("feature_image_dataset_list",feature_image_dataset_list)
            feature_price_dataset_list[column_idx].append(temp_price_list)
            #print("price list",price_list)
            feature_label_dataset_list.append(temp_label_list)
            column_idx += 1

        print("Final len images",len(feature_image_dataset_list),
            "len image list index (i.e. feature) 0",len(feature_image_dataset_list[0][0]))
        print("Final len price list",len(feature_price_dataset_list),
            "len feature_price_dataset_list index 0 (i.e. column 0)", len(feature_price_dataset_list[0][0]),feature_price_dataset_list)
        print("Final len labels", len(feature_label_dataset_list),feature_label_dataset_list) # 2455=total range*5
        
        feature_image_dataset_list = np.array(feature_image_dataset_list) 
        #print("Final Shape of images before transpose:", feature_image_dataset_list.shape)
        
        #transpose image for CNN
        #(5, 1, 491, 1, 4, 4)
        feature_image_dataset_list= np.transpose(feature_image_dataset_list, (1, 3, 0, 2, 4, 5))
        print("Final Shape of images after transpose:", feature_image_dataset_list.shape)

        return feature_image_dataset_list, feature_price_dataset_list, feature_label_dataset_list

#Generate images from dataset
gaf_method="summation"
sample_range = (0, 1)
image_size = 2 #(x,y)
feature_image_dataset_list, feature_price_dataset_list, feature_label_dataset_list = generate_multiple_feature_images(dataset, image_size=image_size, method=gaf_method, sample_range=sample_range)

print("shape [0] set",np.array(feature_image_dataset_list[0]).shape)

np.set_printoptions()

full_feature_num_samples 524
len dataset 524 num_images_to_generate 131
data in GAF [6.64530029e+02 6.49349976e+02 6.65380005e+02 6.59849976e+02
 6.65559998e+02 6.70489990e+02 6.66510010e+02 6.59090027e+02
 6.67719971e+02 6.83890015e+02 6.90330017e+02 6.96080017e+02
 6.99919983e+02 7.05270020e+02 7.05140015e+02 7.53119995e+02
 7.46010010e+02 7.45320007e+02 7.08150024e+02 7.10489990e+02
 7.17400024e+02 7.27989990e+02 7.40440002e+02 7.55030029e+02
 7.51179993e+02 7.47520020e+02 7.42049988e+02 7.50469971e+02
 7.41919983e+02 7.41969971e+02 7.43900024e+02 7.47989990e+02
 7.54650024e+02 7.45260010e+02 7.44969971e+02 7.27789978e+02
 7.41200012e+02 7.37309998e+02 7.40450012e+02 7.14190002e+02
 7.24500000e+02 6.92330017e+02 6.75590027e+02 7.12200012e+02
 6.74570007e+02 6.93390015e+02 7.16780029e+02 7.10210022e+02
 7.06059998e+02 7.03919983e+02 6.75599976e+02 6.76679993e+02
 6.99530029e+02 6.82289978e+02 6.54010010e+02 6.37640015e+02
 6.65260010e+02 6.72690002e+02 6.74760010e+02 6.89619995e+02
 

ValueError: Expected 2D array, got 1D array instead:
array=[6.64530029e+02 6.49349976e+02 6.65380005e+02 6.59849976e+02
 6.65559998e+02 6.70489990e+02 6.66510010e+02 6.59090027e+02
 6.67719971e+02 6.83890015e+02 6.90330017e+02 6.96080017e+02
 6.99919983e+02 7.05270020e+02 7.05140015e+02 7.53119995e+02
 7.46010010e+02 7.45320007e+02 7.08150024e+02 7.10489990e+02
 7.17400024e+02 7.27989990e+02 7.40440002e+02 7.55030029e+02
 7.51179993e+02 7.47520020e+02 7.42049988e+02 7.50469971e+02
 7.41919983e+02 7.41969971e+02 7.43900024e+02 7.47989990e+02
 7.54650024e+02 7.45260010e+02 7.44969971e+02 7.27789978e+02
 7.41200012e+02 7.37309998e+02 7.40450012e+02 7.14190002e+02
 7.24500000e+02 6.92330017e+02 6.75590027e+02 7.12200012e+02
 6.74570007e+02 6.93390015e+02 7.16780029e+02 7.10210022e+02
 7.06059998e+02 7.03919983e+02 6.75599976e+02 6.76679993e+02
 6.99530029e+02 6.82289978e+02 6.54010010e+02 6.37640015e+02
 6.65260010e+02 6.72690002e+02 6.74760010e+02 6.89619995e+02
 6.81070007e+02 6.83190002

In [ ]:
# np.set_printoptions(threshold=np.inf)

# #cols_used = ["Open", "High", "Low", "Close", "Adj Close"]
# cols_used = ["Close"]
# cols_used_count = sum(column_name in cols_used for column_name in dataset.columns)

# #TODO: Instead of insert a list in each list for each feature, insert the elements directly
# def generate_multiple_feature_images(dataset, image_size=32, method="summation", sample_range = (0, 1)):
    
#     feature_image_dataset_list=[[] for _ in range(len(cols_used))]
#     feature_price_dataset_list=[[] for _ in range(len(cols_used))] #="Open", "High", "Low", "Close" , "Adj Close"
#     feature_label_dataset_list=[] #next value for each chunk of ="Open", "High", "Low", "Close" , "Adj Close"
#     column_idx = 0

#     for idx, column_name in enumerate(dataset.columns):

#       #create open, high, low and close images
#       #if column_name in ["Open", "High", "Low", "Close", "Adj Close"]:
#       if column_name in ["Close"]:
#         temp_image_list = []
#         temp_price_list = []
#         temp_label_list = []
#         #print("dataset idx", idx, "len rows this data feature", len(dataset[i]), "dataset[i].shape", dataset[i].shape, "dataset i:", dataset[i])

#         full_feature_data = dataset[column_name].values
#         full_feature_num_samples = len(full_feature_data)
#         print("full_feature_num_samples",full_feature_num_samples)

#         num_windows = image_size
#         #add 1 for last window label
#         window_size = full_feature_num_samples - (image_size + 1)

#         #loop by data_chunk so each chunk represents the price series that we slide by one day forward
#         # TODO: parallelism
#         for curr_window_index in range(num_windows):
          
#           curr_sliding_window_data = full_feature_data[curr_window_index:window_size+curr_window_index]

#           #print(f"Curr window len {len(curr_sliding_window_data)} first value {curr_sliding_window_data[0]} next value {curr_sliding_window_data[1]} last value {curr_sliding_window_data[490]}")
          
#           target_num_chunks = floor(window_size / image_size)
#           #print(f"Target number of chunks for curr Window {column_name}",target_num_chunks)
          
#           for cur_chunk in range(target_num_chunks):
#             #print("cur_chunk",cur_chunk)
            
#             #chunk size of image size
#             data_chunk = curr_sliding_window_data[cur_chunk*image_size:(cur_chunk*image_size)+image_size]
#             #if (cur_chunk == 31 or cur_chunk == 32):
#               #print("cur_chunk",cur_chunk,"data chunk",data_chunk)
#             #append gaf image to image list. store price feature values in price list
#             gaf_images, price_list = generate_gaf_images(data_chunk, gaf_img_sz=image_size, method=method, sample_range=sample_range)
#             temp_image_list.append(gaf_images)
#             #***print("At chunk",cur_chunk,"input chunk size",len(data_chunk),"shape gaf images",gaf_images.shape, "len temp image list",len(temp_image_list))
            
#             # if(cur_chunk==0):
#             #   print("Price Data Pre-Gaf: i", cur_chunk, "len",len(data_chunk), "shape", feature_data.shape, "data",data_chunk)
#             #   print("Image Returned: idx", idx, "image size", gaf_images.size, f"first {image_size} image vals", gaf_images.flatten()[:image_size])
            
#             temp_price_list.append(price_list)
#             #print("At chunk",cur_chunk,"input chunk size",len(data_chunk),"len price_list",len(price_list),price_list)
            
#             #get next single value after the chunk as label to list
#             #print("appending to temp label list",feature_data[cur_chunk + image_size + 1])
#             temp_label_list.append(full_feature_data[cur_chunk + image_size + 1])
#             #feature_label_index_dataset_list.append(feature_data[cur_chunk + image_size + 1])
#             #print("chunk",cur_chunk,"label for",column_name,"price",feature_data[cur_chunk + image_size + 1])
#             #if(column_name == "Open"):
#               #index position for the label of this chunk
#               #feature_label_index_dataset_list.append(cur_chunk + image_size + 1)
#               #print("at chunk",cur_chunk,"feature label list",feature_label_index_dataset_list)
        
#         print(f"Column {column_name} temp image list len to append",len(temp_image_list))
#         feature_image_dataset_list[column_idx].append(temp_image_list)
#         #print("feature_image_dataset_list",feature_image_dataset_list)
#         feature_price_dataset_list[column_idx].append(temp_price_list)
#         #print("price list",price_list)
#         feature_label_dataset_list.append(temp_label_list)
#         column_idx += 1

#     # print("Final len images",len(feature_image_dataset_list),
#     #        "len image list index (i.e. feature) 0",len(feature_image_dataset_list[0][0]))
#     # print("Final len price list",len(feature_price_dataset_list),
#     #        "len feature_price_dataset_list index 0 (i.e. column 0)", len(feature_price_dataset_list[0][0]),feature_price_dataset_list)
#     # print("Final len labels", len(feature_label_dataset_list),feature_label_dataset_list) # 2455=total range*5
    
#     feature_image_dataset_list = np.array(feature_image_dataset_list) 
#     #print("Final Shape of images before transpose:", feature_image_dataset_list.shape)
    
#     #transpose image for CNN
#     #(5, 1, 491, 1, 32, 32)
#     feature_image_dataset_list= np.transpose(feature_image_dataset_list, (1, 3, 0, 2, 4, 5))
#     #print("Final Shape of images after transpose:", feature_image_dataset_list.shape)

#     return feature_image_dataset_list, feature_price_dataset_list, feature_label_dataset_list

# #Generate images from dataset
# gaf_method="summation"
# sample_range = (0, 1)
# image_size = 32 #(x,y)
# feature_image_dataset_list, feature_price_dataset_list, feature_label_dataset_list = generate_multiple_feature_images(dataset, image_size=image_size, method=gaf_method, sample_range=sample_range)

# print("shape [0] set",np.array(feature_image_dataset_list[0]).shape)

# np.set_printoptions()

full_feature_num_samples 524
Column Close temp image list len to append 480
shape [0] set (1, 1, 480, 32, 32)


Initialize image arrays

In [ ]:
images_array=np.array(feature_image_dataset_list)
prices_array=np.array(feature_price_dataset_list) #not used
labels_array=np.array(feature_label_dataset_list)

Set inputs

In [ ]:
batch_size =5
num_epochs_input = 100

In [ ]:
#squeeze arrays
images_array = images_array.squeeze(axis=(0, 1))
print("len img",len(images_array),"image shape",images_array.shape)#,"prices_array[0][0]",prices_array[0][0])
print("len label",len(labels_array),"labels shape",labels_array.shape)#,"prices_array[0][0]",prices_array[0][0])

len img 1 image shape (1, 480, 32, 32)
len label 1 labels shape (1, 480)


## Prepare Training/Testing Datasets for Neural Network

Apply Transform to image to convert to tensor and normalize

In [ ]:
def SetTransform(normalize_ftor=0.5,resolution_x=32,resolution_y=32):
    return transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([normalize_ftor], [normalize_ftor])
    #transforms.Resize((resolution_x, resolution_y))
    ])

In [ ]:
transform = SetTransform(normalize_ftor=0.5, resolution_x=32, resolution_y=32)

In [ ]:
feature_image_dataset_list_f32 = np.array(images_array).astype(np.float32)
feature_image_dataset_list_f32 = feature_image_dataset_list_f32.reshape(-1, image_size, image_size)
#images_array = np.transpose(feature_image_dataset_list, (1, 0, 2, 3))

#scaler = MinMaxScaler()
scaler = StandardScaler()
labels_array = np.array(labels_array)#.astype(np.float32)
#print("labels array",labels_array)
reshaped_labels_array = labels_array.reshape(-1, 1)
#print("reshaped labels array",reshaped_labels_array)
labels_scaled_list_f32 = scaler.fit_transform(reshaped_labels_array).reshape(-1,).astype(np.float32)
#print("scaled labels",labels_scaled_list_f32)
print("4D image array shape",images_array.shape)
print("3D reshaped image array ",feature_image_dataset_list_f32.shape)
print("labels shape",reshaped_labels_array.shape)

4D image array shape (1, 480, 32, 32)
3D reshaped image array  (480, 32, 32)
labels shape (480, 1)


Prepare features data (close, high, low, etc) for Dataloader

In [ ]:
class DataPrep(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs #all features in one large array
        self.labels = labels
        self.transform = SetTransform()

    def __len__(self):
      return len(self.inputs)

    def __getitem__(self, index):
        X = self.inputs[index]
        Y = self.labels[index]
        return X, Y
  
    def prepare_ordered_dataset(self):
        x = []
        y = []
        #print("len inputs", len(self.inputs), "shape", self.inputs.shape, self.inputs.shape[0])
        #print("len images 0",len(self.inputs), "len images 0:",len(self.inputs[0]))
        #print("images 0:",self.inputs[0])
        #print("labels",self.labels)
        #print("len labels", len(self.labels), self.labels.shape, self.labels.shape[0])
        
        np.set_printoptions()
        for image_num in range(self.inputs.shape[0]):
            #print("len image data 0",len(self.inputs[data_window][0]),"shape",self.inputs[data_window].shape)
            #print("label data",self.labels[image_num][0])
            #print("imag num:",image_num)
            #print("image data at index image_num len:",len(self.inputs[image_num]))
            
            self.inputs[image_num] = self.transform(self.inputs[image_num])
            
            x.append(np.expand_dims(self.inputs[image_num], axis=0))
            y.append(self.labels[image_num])
            #print("img num",image_num,"label",self.labels[image_num])
            #print("img num",image_num,"img",self.inputs[image_num])
            
        np.set_printoptions(threshold=np.inf)

        #cnn requests labels size (4,1) instead of (4)
        y = np.expand_dims(y, axis=1) 
        #print("size self",self.inputs.shape,self.labels.shape)
        #print("size self",len(x),len(y))
        dataset = [(img, label) for img, label in zip(x, y)]
        #print("type dataset returned",type(dataset), len(dataset), len(dataset[0]), len(dataset[1]))
        #print("len dataset[0][0]",len(dataset[0][0][0][0]))
        #print("len dataset[1][1]",len(dataset[1][1]))
        #print("dataset[0]",dataset[1])
        return dataset
        
        #return np.array(x),np.array(y)
    
    def split_data(self,dataset, batch_size=batch_size, test_size=0.2, train_shuffle=False):
        num_samples = len(dataset)
        print("numsamples",num_samples)
        num_test_samples = int(test_size * num_samples)
        num_train_samples = num_samples - num_test_samples
        print("num_train_samples",num_train_samples)
        indices = np.random.permutation(num_samples)
        train_indices = indices[:num_train_samples]
        test_indices = indices[num_train_samples:]
        #print("len train",len(train_indices),"len test",len(test_indices))

        train_dataset = Subset(dataset, train_indices)
        test_dataset = Subset(dataset, test_indices)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=train_shuffle)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        # train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=train_shuffle)
        # test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

        #for e in train_loader:
            #print("train loader ele",e)

        # sample_batch = next(iter(train_loader))
        # input_shape = sample_batch[0].shape
        # label_shape = sample_batch[1].shape
        # print("input len",len(input_shape),"input shape",input_shape,"label len",len(label_shape))

        return train_loader, test_loader

In [ ]:
def Generate_Train_And_Test_Loaders(feature_image_dataset_list_f32,labels_scaled_list_f32, train_shuffle=False, batch_size=batch_size):

    #print("feature_image_dataset_list_f32[0][0].shape",feature_image_dataset_list_f32[0][0].shape, "feature_image_dataset_list_f32[0][0].shape[0]", feature_image_dataset_list_f32[0][0].shape[0])

    #reshape for cnn
    #reshaped_feature_image_dataset_list_f32 = np.expand_dims(feature_image_dataset_list_f32[0][0].reshape(-1, *feature_image_dataset_list_f32[0][0].shape[2:]), axis=1)
    #print("feature_image_dataset_list_f32 shape",feature_image_dataset_list_f32.shape)
    #print("res",reshaped_feature_image_dataset_list_f32.shape)

    #generate a list for images and labels
    data_prep_class = DataPrep(feature_image_dataset_list_f32, labels_scaled_list_f32)

    #print("feature_image_dataset_list_f32",feature_image_dataset_list_f32[0][0].shape)
    #print("labels_scaled_list_f32",labels_scaled_list_f32.shape)
    #returns list size all observations of all features of size 2:
    #(image32x32,label) i.e. shape (5*480,32,32) and (5*480,1)
    dataset = data_prep_class.prepare_ordered_dataset()

    # for c in range(len(dataset[0])):
    #     print(f"size labels {c}",dataset[1][c].size)
    #     print(f"size image {c}",dataset[0][c].shape)

    batch_size = batch_size
    train_loader, test_loader = data_prep_class.split_data(dataset, 
                                                         batch_size=batch_size,
                                                         train_shuffle=train_shuffle)

    # for c,e in enumerate(train_loader):
    #     print("count",c)
        # print("type",type(e))
        # print("imga",e[0].shape)
        # print("label",e[1].shape)
    #returns 191 train_loaders that contain batch of 10 images32x32 and 10 labels
    #=191*10=1910 i.e. 80% of 2400 total
    return train_loader,test_loader

train_loader,test_loader = Generate_Train_And_Test_Loaders(feature_image_dataset_list_f32,labels_scaled_list_f32, batch_size=batch_size)

numsamples 480
num_train_samples 384


In [ ]:
run_lstm_simple(train_loader)

epoch  0: epoch 0 ind 0 labels[ind] size torch.Size([1]) labels[ind] tensor([1.2315], device='cuda:0')
initial idx 0
Adding to Node Inputs at idx 0 inputs: tensor([[[-3.0000, -2.9769, -2.9901, -2.9763, -2.9596, -2.9735, -2.9915,
          -2.9696, -2.8902, -2.8435, -2.7938, -2.7562, -2.6971, -2.6986,
          -1.3785, -1.8085, -1.8374, -2.6618, -2.6313, -2.5302, -2.3361,
          -2.0140, -1.0000, -1.5349, -1.7405, -1.9603, -1.5812, -1.9648,
          -1.9631, -1.8934, -1.7178, -1.1695],
         [-2.9769, -2.9080, -2.9369, -2.9069, -2.8762, -2.9014, -2.9405,
          -2.8940, -2.7691, -2.7045, -2.6389, -2.5907, -2.5169, -2.5188,
          -1.0763, -1.5217, -1.5522, -2.4738, -2.4369, -2.3171, -2.0949,
          -1.7408, -0.6966, -1.2364, -1.4501, -1.6831, -1.2842, -1.6879,
          -1.6861, -1.6117, -1.4263, -0.8652],
         [-2.9901, -2.9369, -2.9605, -2.9361, -2.9101, -2.9314, -2.9633,
          -2.9253, -2.8159, -2.7573, -2.6971, -2.6524, -2.5835, -2.5853,
          -1.1815, -

C:\Users\Sergio\AppData\Local\Temp\ipykernel_26404\1903641315.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_list = [torch.tensor(y, device=self.lstm_param.device) for y in y_list]


tensor([[[-2.3287e+00, -2.2853e+00, -2.3597e+00, -2.5273e+00, -2.3271e+00,
          -2.2506e+00, -2.0367e+00, -1.8872e+00, -1.8539e+00, -2.8245e+00,
          -2.5501e+00, -2.7977e+00, -2.6732e+00, -2.5544e+00, -2.3971e+00,
          -2.4687e+00, -2.4410e+00, -2.2568e+00, -2.2127e+00, -2.2597e+00,
          -2.2672e+00, -2.1562e+00, -1.4724e+00, -1.3879e+00, -1.0854e+00,
          -1.0394e+00, -1.8064e-01, -1.2441e+00, -1.3197e+00, -1.8699e+00,
          -2.1380e+00, -2.1103e+00],
         [-2.2853e+00, -2.2408e+00, -2.3171e+00, -2.4896e+00, -2.2836e+00,
          -2.2053e+00, -1.9873e+00, -1.8354e+00, -1.8017e+00, -2.8002e+00,
          -2.5132e+00, -2.7718e+00, -2.6411e+00, -2.5177e+00, -2.3555e+00,
          -2.4292e+00, -2.4007e+00, -2.2117e+00, -2.1666e+00, -2.2146e+00,
          -2.2223e+00, -2.1089e+00, -1.4165e+00, -1.3315e+00, -1.0281e+00,
          -9.8204e-01, -1.2868e-01, -1.1871e+00, -1.2630e+00, -1.8179e+00,
          -2.0903e+00, -2.0621e+00],
         [-2.3597e+00, -2.

KeyboardInterrupt: 